In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import cv2
import numpy as np
import toml
import glob
import h5py
import warnings
import pandas as pd

from tqdm.auto import tqdm
from markovids import vid
from qd_analysis.util import clean_df
from scipy import signal
from skimage.measure import regionprops_table
from joblib import Parallel, delayed

In [3]:
segmentation_dir = "_segmentation_tau-0-pretrain"

## User functions

In [4]:
def sos_filter(x, fps, tau=0.01, order=3):
    sos = signal.butter(order, (1 / tau) / (fps / 2), btype="low", output="sos")
    return signal.sosfiltfilt(sos, x, axis=0)


def lp_filter(x, sigma):
    return cv2.GaussianBlur(x, [0, 0], sigma, sigma)


def bp_filter(x, sigma1, sigma2, clip=True):
    if (sigma2 == 0) or (sigma2 is None):
        return lp_filter(x, sigma1)
    else:
        return np.clip(
            lp_filter(x, sigma1) - lp_filter(x, sigma2),
            0 if clip == True else -np.inf,
            np.inf,
        )

In [5]:
def get_average_intensity(
    dat_file,
    spacing=400,
    reader_kwargs={"threads": 2},
    segmentation_dir=segmentation_dir,
    distortion_coeffs=None,
    intrinsic_matrix=None,
    bground_dir="_bground",
    output_dir="_ave_intensity",
    summary_quantiles=[0.99, 0.999, 1.0],
    bpass=(1, 3),
    force=False,
):
    # ONLY undistort at the end so we don't have to account for it
    # when we don't need it (e.g. keypoints)
    metadata = toml.load(os.path.join(os.path.dirname(dat_file), "../metadata.toml"))
    dirname, fname_reflectance = os.path.split(dat_file.replace("fluorescence", "reflectance"))
    fname_reflectance, ext = os.path.splitext(fname_reflectance)
    fname_fluorescence, ext = os.path.splitext(os.path.basename(dat_file))

    segmentation_path = os.path.join(dirname, segmentation_dir, f"{fname_reflectance}.hdf5")
    bground_path = os.path.join(dirname, bground_dir, f"{fname_fluorescence}.hdf5")
    save_file = os.path.join(dirname, output_dir, f"{fname_fluorescence}.parquet")

    os.makedirs(os.path.join(dirname, output_dir), exist_ok=True)
    os.makedirs(os.path.join(dirname, bground_dir), exist_ok=True)

    if os.path.exists(save_file) and not force:
        stats = pd.read_parquet(save_file)
        return stats

    reader = vid.io.AutoReader(
        dat_file,
        **reader_kwargs,
    )
    frame_range = range(0, reader.nframes, spacing)

    if os.path.exists(segmentation_path):
        try:
            with h5py.File(segmentation_path) as f:
                masks = f["labels"][frame_range]
                masks = masks.astype("uint8")
        except Exception as e:
            print(e)
            warnings.warn(f"Could not load mask from {segmentation_path}")
            reader.close()
            return None
    else:
        warnings.warn(f"No mask found {segmentation_path} for {dat_file}")
        reader.close()
        masks = None
        # return None

    if os.path.exists(bground_path):
        with warnings.catch_warnings():
            with h5py.File(bground_path, "r") as f:
                rolling_bgrounds = f["bground"][()]
                idxs = f["frame_idxs"][()]
            frames = reader.get_frames(frame_range)
    else:
        warnings.warn(f"No background found {dat_file}")
        reader.close()
        return None

    bground_sub = np.zeros(frames.shape, dtype="int16")
    for i, (_idx, _frame) in enumerate(zip(frame_range, frames)):
        use_bground = np.argmin(np.abs(idxs - _idx))
        bground_sub[i] = np.clip(_frame - rolling_bgrounds[use_bground], 0, 255)

    bground_sub_raw = bground_sub.copy()
    for i in range(len(bground_sub)):
        bground_sub[i] = bp_filter(bground_sub[i], *bpass)

    if intrinsic_matrix is not None:
        if masks is not None:
            for i in range(len(masks)):
                masks[i] = cv2.undistort(masks[i], intrinsic_matrix, distortion_coeffs)
        for i in range(len(bground_sub)):
            bground_sub[i] = cv2.undistort(bground_sub[i], intrinsic_matrix, distortion_coeffs)
        for i in range(len(bground_sub_raw)):
            bground_sub_raw[i] = cv2.undistort(bground_sub_raw[i], intrinsic_matrix, distortion_coeffs)

    if masks is not None:

        extra_props = []
        for _quantile in summary_quantiles:
            func = lambda regionmask, im, quantile=_quantile: np.quantile(im[regionmask], quantile)
            func.__name__ = f"q{_quantile}"
            extra_props.append(func)

        func = lambda regionmask, im: np.std(im[regionmask])
        func.__name__ = "std"
        extra_props.append(func)

        bground_mu = bground_sub[masks == 0].mean()
        bground_sig = bground_sub[masks == 0].std()

        func = lambda regionmask, im: np.mean((im[regionmask] > (bground_mu + 10 * bground_sig)).astype("float32"))
        func.__name__ = "frac_pixels_above_background"
        extra_props.append(func)

        stats = [
            regionprops_table(
                _mask,
                intensity_image=_im,
                properties=["mean_intensity"],
                extra_properties=extra_props,
            )
            for _im, _mask in zip(bground_sub, masks)
        ]

        dfs = []
        use_idx = []
        for i, (_stat, _idx) in enumerate(zip(stats, frame_range)):
            _df = pd.DataFrame(_stat)
            if _df.empty:
                continue
            _df["frame_idx"] = _idx
            dfs.append(_df)
            use_idx.append(i)
        use_idx = np.array(use_idx).astype("int")
        stats = pd.concat(dfs, ignore_index=True)
    else:
        use_idx = np.arange(len(frames)).astype("int")
        stats = pd.DataFrame()

    for _quantile in summary_quantiles:
        _tmp = np.quantile(bground_sub_raw[use_idx], _quantile, axis=(1, 2))
        stats[f"q{_quantile}_fullframe_raw"] = _tmp
    stats["mean_intensity_fullframe_raw"] = np.mean(bground_sub_raw[use_idx], axis=(1, 2))
    stats["std_intensity_fullframe_raw"] = np.std(bground_sub_raw[use_idx], axis=(1, 2))

    for _quantile in summary_quantiles:
        _tmp = np.quantile(bground_sub[use_idx], _quantile, axis=(1, 2))
        stats[f"q{_quantile}_fullframe_bpass"] = _tmp
    stats["mean_intensity_fullframe_bpass"] = np.mean(bground_sub[use_idx], axis=(1, 2))
    stats["std_intensity_fullframe_bpass"] = np.std(bground_sub[use_idx], axis=(1, 2))

    stats["start_time"] = metadata["start_time"]
    stats["hw_trigger_pulse_width"] = str(metadata["cli_parameters"]["hw_trigger_pulse_width"])
    stats["filename"] = dat_file
    for k, v in metadata["user_input"].items():
        stats[k] = v

    stats.to_parquet(save_file)
    return stats

# Quantify fluorescence decay over time

In [6]:
# need to use different calibration files for different days...

In [7]:
root_dir = os.path.expanduser("~/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/quantum_dots/")

In [8]:
base_dir = os.path.join(root_dir, "timecourse_01")
fluo_files = sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))
base_dir = os.path.join(root_dir, "timecourse_01_agarose_beads")
fluo_files += sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))

In [9]:
calibration_data = toml.load(os.path.join(root_dir, "timecourse_01_calibration.toml"))

In [10]:
# get subject names and filter that stuff...
metadata = {}
for _file in tqdm(fluo_files):
    metadata[_file] = toml.load(os.path.join(os.path.dirname(_file), "../metadata.toml"))

  0%|          | 0/780 [00:00<?, ?it/s]

In [11]:
delays = []
for _file in fluo_files:
    cam = os.path.basename(_file).replace("-fluorescence.avi", "")
    delays.append(
        delayed(get_average_intensity)(
            _file,
            force=False,
            intrinsic_matrix=np.array(calibration_data["intrinsics"][cam]),
            distortion_coeffs=np.array(calibration_data["distortion_coeffs"][cam])        
        )
    )
print(len(delays))
dat = Parallel(n_jobs=-1, verbose=10, backend="multiprocessing")(delays)

780


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.08932232856750488s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.06818342208862305s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  78 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1283891201019287s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 220 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 304 tasks      | elapsed:    1.3s
[Para

In [12]:
base_dir = os.path.join(root_dir, "timecourse_02")
fluo_files = sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))
base_dir = os.path.join(root_dir, "timecourse_02_joints")
fluo_files += sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))
base_dir = os.path.join(root_dir, "timecourse_03")
fluo_files += sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))
base_dir = os.path.join(root_dir, "sciadv_rebuttal/exposure_series")
fluo_files += sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))
base_dir = os.path.join(root_dir, "sciadv_rebuttal/dilution_series")
fluo_files += sorted(glob.glob(os.path.join(base_dir, "**", "Basler*fluorescence.avi"), recursive=True))

In [13]:
calibration_data = [toml.load(os.path.join(root_dir, "timecourse_02_calibration_v1.toml")),
                    toml.load(os.path.join(root_dir, "timecourse_02_calibration_v2.toml")),
                    toml.load(os.path.join(root_dir, "timecourse_04_calibration.toml")),
                    ]

In [14]:
# get subject names and filter that stuff...
metadata = {}
for _file in tqdm(fluo_files):
    metadata[_file] = toml.load(os.path.join(os.path.dirname(_file), "../metadata.toml"))

  0%|          | 0/739 [00:00<?, ?it/s]

In [15]:
delays = []
for _file in fluo_files:
    cam = os.path.basename(_file).replace("-fluorescence.avi", "")
    timestamp = pd.to_datetime(metadata[_file]["start_time"])
    if timestamp.floor("d") > pd.to_datetime("2024-11-10"):
        use_calibration_data = calibration_data[2]
    elif timestamp.floor("d") > pd.to_datetime("2024-06-10"):
        use_calibration_data = calibration_data[1]
    else:
        use_calibration_data = calibration_data[0]
    # for 0610 load v1 after that load v2 calibration data...
    delays.append(
        delayed(get_average_intensity)(
            _file,
            force=False,
            intrinsic_matrix=np.array(use_calibration_data["intrinsics"][cam]),
            distortion_coeffs=np.array(use_calibration_data["distortion_coeffs"][cam])        
        )
    )
print(len(delays))
dat2 = Parallel(n_jobs=-1, verbose=10, backend="multiprocessing")(delays)

739


[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.058277130126953125s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.06664323806762695s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 220 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 288 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.032125059616554s.) Setting batch_size=1.
[Para

In [16]:
dat += dat2

In [17]:
fluo_df = pd.concat(dat, ignore_index=True)

# clean up df and save off

In [18]:
config = toml.load("config.toml")

In [19]:
fluo_df = clean_df(
    fluo_df,
    exp_types=config["aliases"],
    subject_typos=config["typos"]["subject"],
    chk_fields=config["parse_metadata"]["chk_fields"],
    exclude_subjects=config["exclusions"]["subjects"],
    exclude_dates=config["exclusions"]["dates"],
    exclude_pairs=config["exclusions"]["pairs"],
)

In [20]:
use_cameras = list(toml.load(os.path.join(root_dir, "timecourse_01_calibration.toml"))["distortion_coeffs"].keys())
fluo_df = fluo_df.query("camera.isin(@use_cameras)").copy()

In [21]:
os.makedirs(config["dirs"]["analysis"], exist_ok=True)
fluo_df.to_parquet(os.path.join(config["dirs"]["analysis"], "fluorescence_intensity_over_time.parquet"))